In [1]:
import numpy as np
import pandas as pd

from math import sqrt
from pathlib import Path
from tqdm import tqdm
tqdm.pandas()

from datetime import datetime, timedelta
import gc

In [3]:
!nvidia-smi  #check GPU

Thu Feb  9 17:11:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [ ]:
import sys
!cp ../input/rapids/rapids.21.06 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

import cudf

In [4]:
data_path = Path('../input/h-and-m-personalized-fashion-recommendations/')
N = 12

### Read the transactions data
##### article_id: number => dtypes: int32
##### customer_id: hex => change to int64 (reduct memory, enhance speed)
##### read, modify dataframe by cudf (GPU)

In [ ]:
# df  = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv',
#                             usecols= ['t_dat', 'customer_id', 'article_id'], 
#                             dtype={'article_id': 'int32', 't_dat': 'string', 'customer_id': 'string'})
# df ['customer_id'] = df ['customer_id'].str[-16:].str.hex_to_int().astype('int64')
    
# df['t_dat'] = cudf.to_datetime(df['t_dat'])
# last_ts = df['t_dat'].max()

df = pd.read_csv(data_path / 'transactions_train.csv',
                 usecols = ['t_dat', 'customer_id', 'article_id'],
                 dtype={'article_id': 'str'})
#change -> int32 => need to add '0' in the first of article_id
df['t_dat'] = pd.to_datetime(df['t_dat'])
last_ts = df['t_dat'].max()

### Add the last day of billing week

In [9]:
# %%time       #check time GPU
# ##add the last day of billing week of each transactions (previous weeks from here 23-9-2020 Tuesday)
# #df['ldbw'] = df['t_dat'].progress_apply(lambda d: last_ts - (last_ts - d).floor('7D'))
# ##change -> day of week -> ldbw = t_dat - dow + 1 
# ##the last day of week = Tuesday -> dow >= 2 (>= Wednesday): ldbw + 7 days
# tmp = df[['t_dat']].copy().to_pandas()
# tmp['dow'] = tmp['t_dat'].dt.dayofweek
# tmp['ldbw'] = tmp['t_dat'] - pd.TimedeltaIndex(tmp['dow'] - 1, unit='D')
# tmp.loc[tmp['dow'] >=2 , 'ldbw'] = tmp.loc[tmp['dow'] >=2 , 'ldbw'] + pd.TimedeltaIndex(np.ones(len(tmp.loc[tmp['dow'] >=2])) * 7, unit='D')

# df['ldbw'] = tmp['ldbw'].values


tmp = df[['t_dat']].copy()
tmp['dow'] = tmp['t_dat'].dt.dayofweek
tmp['ldbw'] = tmp['t_dat'] - pd.TimedeltaIndex(tmp['dow'] - 1, unit='D')
tmp.loc[tmp['dow'] >=2 , 'ldbw'] = tmp.loc[tmp['dow'] >=2 , 'ldbw'] + pd.TimedeltaIndex(np.ones(len(tmp.loc[tmp['dow'] >=2])) * 7, unit='D')

df['ldbw'] = tmp['ldbw'].values



### Count the number of transactions per week 

In [10]:
weekly_sales = df.drop('customer_id', axis=1).groupby(['ldbw', 'article_id']).count().reset_index()
weekly_sales = weekly_sales.rename(columns={'t_dat': 'count'})

In [ ]:
print(weekly_sales.head())
print(weekly_sales.tail())

In [12]:
df = df.merge(weekly_sales, on=['ldbw', 'article_id'], how = 'left')

In [9]:
df.tail()

,t_dat,customer_id,article_id,ldbw,count
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,2020-09-22,64
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,2020-09-22,36
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,2020-09-22,6
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,2020-09-22,12
31788323,2020-09-22,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,898573003,2020-09-22,251


### Let's assume that in the target week sales will be similar to the last week of the training data

In [14]:
weekly_sales = weekly_sales.reset_index().set_index('article_id')

df = df.merge(
    weekly_sales.loc[weekly_sales['ldbw']==last_ts, ['count']],
    on='article_id', suffixes=("", "_targ"))
#filter all article_id which purchased in the last week (in all history)
df['count_targ'].fillna(0, inplace=True)
#count_target: count in the last week (truong hop count 2 cai bang nhau ->quotient = 0?)
del weekly_sales

In [15]:
# print(df.loc[df['count'] == 6, ['count', 'count_targ']])
# print(df['count_targ'].value_counts())
# print(df['count'].value_counts())
df.count()

t_dat          12541136
customer_id    12541136
article_id     12541136
ldbw           12541136
count_x        12541136
index          12541136
count_y        12541136
count          12541136
count_targ     12541136
dtype: int64

### Calculate sales rate adjusted for changes in product popularity 

In [16]:
df['quotient'] = df['count_targ'] / df['count']

number transac each article_id in the last week / number transac each article_id each previous week

### Take supposedly popular products

In [17]:
target_sales = df.drop('customer_id', axis=1).groupby('article_id')['quotient'].sum()
general_pred = target_sales.nlargest(20).index.tolist()
del target_sales
#target_sales.nlargest(20).to_csv("trending_weekly.csv")

In [18]:
general_pred

[706016001,
 706016002,
 372860001,
 610776002,
 759871002,
 464297007,
 372860002,
 610776001,
 399223001,
 706016003,
 720125001,
 156231001,
 562245046,
 562245001,
 351484002,
 399256001,
 673396002,
 568601006,
 448509014,
 673677002]

-> article_id :
lượng mua tuần này/ lần lượt tất cả các tuần (trước) của article_id đó càng lớn

-> tổng: độ phổ biến tính đến tuần trước của món hàng đó #? những sản phẩm mới ra mắt tuần này

### Fill in purchase dictionary
Avoi looping, use df instead of dictionary, manipulate with series


In [ ]:
purchase_dict = {}

for i in tqdm(df.index):
    cust_id = df.at[i, 'customer_id']
    art_id = df.at[i, 'article_id']
    t_dat = df.at[i, 't_dat']

    if cust_id not in purchase_dict:
        purchase_dict[cust_id] = {}

    if art_id not in purchase_dict[cust_id]:
        purchase_dict[cust_id][art_id] = 0
    
    x = max(1, (last_ts - t_dat).days)

    a, b, c, d = 2.5e4, 1.5e5, 2e-1, 1e3
    y = a / np.sqrt(x) + b * np.exp(-c*x) - d

    value = df.at[i, 'quotient'] * max(0, y) # độ phổ biến * tỉ lệ mua lại món hàng đó
    purchase_dict[cust_id][art_id] += value  #moi lan mua tang ti le mua lai mon do do ?

### Make a submission

In [ ]:
sub = pd.read_csv(data_path / 'sample_submission.csv')

pred_list = []
for cust_id in tqdm(sub['customer_id']):
    if cust_id in purchase_dict:
        series = pd.Series(purchase_dict[cust_id])
        series = series[series > 0]
        l = series.nlargest(N).index.tolist()
        if len(l) < N:
            l = l + general_pred[:(N-len(l))]
    else:
        l = general_pred
    pred_list.append(' '.join(l))

sub['prediction'] = pred_list
sub.to_csv('submission.csv', index=None)